# Bayesian Optimization
Idea: Use bayesian optimization for (wrapper) feature selection.

In [35]:
from sklearn import datasets
from bayes_opt import BayesianOptimization

## 1. Import datasets

In [58]:
# regression dataset
boston_data, boston_targets = datasets.load_boston(return_X_y=True)

# classification dataset
cancer_data, cancer_targets = datasets.load_breast_cancer(return_X_y=True)

# define which dataset should be used
data = cancer_data
target_matrix = cancer_targets

#print(data)
#print(target_matrix)

## 2. Specify target function to be optimized

In [66]:
# iterate through data array in order to find index, then return target value for this index
def target_function(*arguments):
    for i in range(0,len(data)):
        if (arguments==data[i]).all():
            return target_matrix[i]


## 3. Actual Bayesian optimization

In [ ]:
# TODO